In [3]:
include("../helperFunctions.jl");
include("../alpcah.jl")
using BenchmarkTools
using HePPCAT
using LinearAlgebra

In [4]:
D = 100;
d = 10;
v = [0.1, 100];
N = [20, 490];
U = generateSubspace(D, d)
Y = generateData(U, v, N);
varianceVector = vcat(v[1]*ones(N[1]), v[2]*ones(N[2]));

In [5]:
using HePPCAT
YL = [];
for i = 1:size(Y)[2]
    push!(YL,Y[:,i])
end
res = heppcat(YL,d,1000)
U_HEPPCAT = svd(res.F).U[:,1:d];
affinityError(U,U_HEPPCAT)

0.4625670374327324

In [25]:
U_test,X_test,v_test = ALPCAH(Y, d; methodType=:ALPCAH_ADMM, alpcahIter= 1000, apgdIter = 5, λr=20, μ=0.25, ρ=1.0)
affinityError(U,U_test)

0.5105453370181829

In [ ]:
xogm, _ = pogm_restart(X0, (x) -> 0, f_grad_smooth_local, f_L_smooth_local;mom = :pogm, niter=iter, fun=fun_nuclear, f_mu=0.01);

In [ ]:
function smoothALPCAH(Y::Matrix, rank::Int, λr::Real; alpcahIter::Int = 200, apgdIter::Int = 5, varianceMethod::Symbol = :groupless, varfloor::Real=1e-9)
    D, N = size(Y)
    v = zeros(N)
    # Krylov-based Lanczos Algorithm
    U = tsvd(Y, rank)[1]
    X = U*U'*Y
    #variance method initialization
    if varianceMethod === :groupless
        v = grouplessVarianceUpdate(Y, X; varfloor=varfloor)
    else
        #v = groupedVarianceUpdate(Y, L*R' ; varfloor=varfloor)
    end
    Π = v.^-1
    Lf = maximum(Π)
    Π = Diagonal(Π)
    X = zeros(size(Y)) # weirdly necessary to get lower error
    grad = K -> -1*(Y-K)*Π
    #alpha = LinRange(0.0, λr, alpcahIter)
    for i=1:alpcahIter
        # apgd
        g_prox = (z,c) -> TSVT(z, c*λr, rank) # TSVT(z, c*alpha[i], rank)
        X, _ = pogm_restart(X, x -> 0, grad, Lf ; mom=:fpgm, g_prox, niter=apgdIter);
        # variance updates
        if varianceMethod === :groupless
            v = grouplessVarianceUpdate(Y, X; varfloor=varfloor)
        else
            #v = groupedVarianceUpdate(Y, L*R' ; varfloor=varfloor)
        end
        Π = v.^-1
        Lf = maximum(Π)
        Π = Diagonal(Π)
    end
    # extract left vectors from X
    U = tsvd(X, rank)[1]
    return U
end